### Behaviour dimensionalitet
136200 readings x 18 features -> 5 readings svarer til 1 epoch fordi det er lidt mærkeligt stillet op..
136200/93/1500 - > cirka 300 epochs per pers med nogle slettede

### FG dimensionalitet
92 x 19 features.. Der er en person der mangler her

### Force dimensionalitet




In [2]:
import mne
import pandas as pd
import numpy as np
import os
import pickle

# Define paths
folder_path = "/Users/maltelau/Desktop/LaBraM-MMDTU/LaBraM-MMDTU/DTUDATA/FG_Data/PreprocessedEEGData"
force_data_path = "/Users/maltelau/Desktop/LaBraM-MMDTU/LaBraM-MMDTU/DTUDATA/FG_Data/Force_df_v2.pkl"
overview_path = "/Users/maltelau/Desktop/LaBraM-MMDTU/LaBraM-MMDTU/DTUDATA/FG_Data/FG_overview_df_v2.pkl"
behavior_path = "/Users/maltelau/Desktop/LaBraM-MMDTU/LaBraM-MMDTU/DTUDATA/FG_Data/Beh_feat_df_v2.pkl"

# Load all data sources
with open(force_data_path, "rb") as f:
    force_df = pickle.load(f)
with open(overview_path, "rb") as f:
    overview_df = pickle.load(f)
with open(behavior_path, "rb") as f:
    behavior_df = pickle.load(f)

# Initialize lists to store data
all_data = []

# Get list of all .fif files
fif_files = [f for f in os.listdir(folder_path) if f.endswith('.fif')]

t=0
# Process each file
for fif_file in fif_files:
    t+=1
    if t==11:
        break
    # Extract participant ID from filename
    participant_id = fif_file.split('_')[0]
    triad_id = int(participant_id[:3])
    
    # Get participant overview data
    participant_overview = overview_df[overview_df['Exp_id'] == participant_id].iloc[0]
    
    # Load epochs
    epochs = mne.read_epochs(os.path.join(folder_path, fif_file), preload=True)
    eeg_data = epochs.get_data()
    
    # Get event IDs to condition mapping
    event_id_to_condition = {v: k for k, v in epochs.event_id.items()}
    
    # For each epoch
    for epoch_idx in range(len(epochs)):
        # Get condition for this epoch
        event_id = epochs.events[epoch_idx][2]
        condition = event_id_to_condition[event_id]
        trial_number = epoch_idx + 1  # 1-based trial number
        
        # Get corresponding force data
        force_data = force_df[
            (force_df['Triad_id'] == triad_id) & 
            (force_df['Trial'] == trial_number) &
            (force_df['Condition'] == condition)
        ]
        
        # Get behavioral features
        beh_data = behavior_df[
            (behavior_df['Triad_id'] == triad_id) &
            (behavior_df['Trial'] == trial_number) &
            (behavior_df['Condition'] == condition)
        ]
        
        if len(force_data) > 0 and len(beh_data) > 0:
            # Aggregate behavioral features
            beh_features = beh_data.pivot(
                columns='Feature', 
                values='Value'
            ).iloc[0]
            
            # Create row for each epoch
            row_data = {
                # Participant information
                'participant_id': participant_id,
                'triad_id': triad_id,
                'subject_id': participant_overview['Subject_id'],
                'friend_status': participant_overview['Friend_status'],
                'age': participant_overview['Age'],
                'gender': participant_overview['Gender'],
                'class_friends': participant_overview['Class_friends'],
                'class_close_friends': participant_overview['Class_close_friends'],
                'friends': participant_overview['Friends'],
                
                # Trial information
                'epoch_idx': epoch_idx,
                'trial_number': trial_number,
                'condition': condition,
                'block': force_data['Block'].iloc[0],
                
                # Device information
                'eeg_device': participant_overview['EEG_device'],
                'force_device': participant_overview['Force_device'],
                'force_port': participant_overview['Force_port'],
                
                # Force data
                'force_level': force_data['Force_level'].iloc[0],
                'target_force': force_data['TargetForce'].iloc[0],
                'force': force_data['Force'].iloc[0],
                'dforce': force_data['dForce'].iloc[0],
                'trial_time': force_data['Trial_time'].iloc[0],
                
                # Behavioral features
                'latency': beh_features['Latency'],
                'stability': beh_features['Stability'],
                'success': beh_features['Success'],
                'relative_force': beh_features['RelativeForce'],
                'cv_variability': beh_features['cVariability'],
                
                # EEG data
                'eeg_data': eeg_data[epoch_idx]
            }
            all_data.append(row_data)

# Create DataFrame
combined_df = pd.DataFrame(all_data)

# Print basic information about the DataFrame
print("DataFrame shape:", combined_df.shape)
print("\nColumns:", combined_df.columns.tolist())
print("\nParticipants:", combined_df['participant_id'].nunique())
print("Total epochs:", len(combined_df))

Reading /Users/maltelau/Desktop/LaBraM-MMDTU/LaBraM-MMDTU/DTUDATA/FG_Data/PreprocessedEEGData/326A_FG_preprocessed-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    5498.00 ms
        0 CTF compensation matrices available
Not setting metadata
298 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/maltelau/Desktop/LaBraM-MMDTU/LaBraM-MMDTU/DTUDATA/FG_Data/PreprocessedEEGData/311A_FG_preprocessed-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    5498.00 ms
        0 CTF compensation matrices available
Not setting metadata
300 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/maltelau/Desktop/LaBraM-MMDTU/LaBraM-MMDTU/DTUDATA/FG_Data/PreprocessedEEGData/318B_FG_preprocessed-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    5498.00 ms
        0 CTF compensation matrices available
Not setting metadata
298 matching events found
N

In [23]:
#find the minimum number of epochs per participant and print which one it is
min_epochs = combined_df.groupby('participant_id').size().min()
min_participant = combined_df.groupby('participant_id').size().idxmin()
print("Participant with minimum epochs:", min_participant)
print("Minimum epochs per participant:", min_epochs)

Participant with minimum epochs: 315A
Minimum epochs per participant: 184


In [24]:
combined_df[combined_df['participant_id'] == min_participant]

,participant_id,triad_id,subject_id,friend_status,age,gender,class_friends,class_close_friends,friends,epoch_idx,...,target_force,force,dforce,trial_time,latency,stability,success,relative_force,cv_variability,eeg_data
837,315A,315,5002,Yes,23.019849,F,12,4,17,0,...,32,0.024422,-31.975578,0.0,3.36,NaN,NaN,NaN,NaN,"[[-3.2322227525583003e-06, -2.877209908547229e..."
838,315A,315,5002,Yes,23.019849,F,12,4,17,1,...,24,0.000001,-23.999999,0.0,2.08,NaN,NaN,NaN,NaN,"[[2.6612269721226767e-06, 3.196427087459597e-0..."
839,315A,315,5002,Yes,23.019849,F,12,4,17,2,...,16,0.012211,-15.987789,0.0,2.08,NaN,NaN,NaN,NaN,"[[1.4575608702216414e-06, 9.931497970683267e-0..."
840,315A,315,5002,Yes,23.019849,F,12,4,17,3,...,28,0.052447,-27.947553,0.0,3.04,NaN,NaN,NaN,NaN,"[[1.8159370256398688e-06, 3.062983068957692e-0..."
841,315A,315,5002,Yes,23.019849,F,12,4,17,4,...,20,0.003154,-19.996846,0.0,2.68,NaN,NaN,NaN,NaN,"[[1.5775243582538678e-06, 1.839377205214987e-0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016,315A,315,5002,Yes,23.019849,F,12,4,17,266,...,16,0.013662,-15.986338,0.0,NaN,NaN,NaN,NaN,NaN,"[[1.1712679679476423e-06, 1.095663265004987e-0..."
1017,315A,315,5002,Yes,23.019849,F,12,4,17,270,...,16,0.000001,-15.999999,0.0,2.16,NaN,NaN,NaN,NaN,"[[-2.6202826575172367e-06, -2.164349552913336e..."
1018,315A,315,5002,Yes,23.019849,F,12,4,17,271,...,24,0.003154,-23.996846,0.0,1.56,NaN,NaN,NaN,NaN,"[[-6.052440681969529e-08, 9.480745433165794e-0..."
1019,315A,315,5002,Yes,23.019849,F,12,4,17,275,...,30,0.003154,-29.996846,0.0,0.92,NaN,NaN,NaN,NaN,"[[4.107268694042432e-07, 4.7403352709807223e-0..."


In [45]:
combined_df[combined_df["participant_id"] == '315A']

,participant_id,triad_id,subject_id,friend_status,age,gender,class_friends,class_close_friends,friends,epoch_idx,...,target_force,force,dforce,trial_time,latency,stability,success,relative_force,cv_variability,eeg_data
837,315A,315,5002,Yes,23.019849,F,12,4,17,0,...,32,0.024422,-31.975578,0.0,3.36,NaN,NaN,NaN,NaN,"[[-3.2322227525583003e-06, -2.877209908547229e..."
838,315A,315,5002,Yes,23.019849,F,12,4,17,1,...,24,0.000001,-23.999999,0.0,2.08,NaN,NaN,NaN,NaN,"[[2.6612269721226767e-06, 3.196427087459597e-0..."
839,315A,315,5002,Yes,23.019849,F,12,4,17,2,...,16,0.012211,-15.987789,0.0,2.08,NaN,NaN,NaN,NaN,"[[1.4575608702216414e-06, 9.931497970683267e-0..."
840,315A,315,5002,Yes,23.019849,F,12,4,17,3,...,28,0.052447,-27.947553,0.0,3.04,NaN,NaN,NaN,NaN,"[[1.8159370256398688e-06, 3.062983068957692e-0..."
841,315A,315,5002,Yes,23.019849,F,12,4,17,4,...,20,0.003154,-19.996846,0.0,2.68,NaN,NaN,NaN,NaN,"[[1.5775243582538678e-06, 1.839377205214987e-0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016,315A,315,5002,Yes,23.019849,F,12,4,17,266,...,16,0.013662,-15.986338,0.0,NaN,NaN,NaN,NaN,NaN,"[[1.1712679679476423e-06, 1.095663265004987e-0..."
1017,315A,315,5002,Yes,23.019849,F,12,4,17,270,...,16,0.000001,-15.999999,0.0,2.16,NaN,NaN,NaN,NaN,"[[-2.6202826575172367e-06, -2.164349552913336e..."
1018,315A,315,5002,Yes,23.019849,F,12,4,17,271,...,24,0.003154,-23.996846,0.0,1.56,NaN,NaN,NaN,NaN,"[[-6.052440681969529e-08, 9.480745433165794e-0..."
1019,315A,315,5002,Yes,23.019849,F,12,4,17,275,...,30,0.003154,-29.996846,0.0,0.92,NaN,NaN,NaN,NaN,"[[4.107268694042432e-07, 4.7403352709807223e-0..."


In [38]:
#find the column "condition"
combined_df["condition"]

0        T23P
1        T23P
2        T23P
3        T23P
4        T23P
        ...  
2738    T13Pn
2739    T13Pn
2740    T13Pn
2741    T13Pn
2742    T13Pn
Name: condition, Length: 2743, dtype: object